In [247]:
#@title Layers Tunable param/ Mapping function
import math
import numpy as np
from typing import List, Tuple
import matplotlib.pyplot as plt
# # Enable interactive tables once at the top
from google.colab import data_table
from IPython.display import HTML

def calc_tunable_params(weights, X = 128):
  xbars=[]
  params=[]
  IFMS=[]
  OFMS=[]
  for each in weights:
    IFM=each[0]*each[1]*each[2]
    param = each[2]*each[3]*each[4]*each[5]  #### check no of parameters ? shouldnt they be in different column
    xbar = math.ceil(each[3]*each[4]*each[2]/ X) * math.ceil(each[5]/ X)
    OFM = each[0]*each[1]*each[5]
    params.append(param)
    xbars.append(xbar)
    IFMS.append(IFM)
    OFMS.append(OFM)
  return params,xbars,IFMS,OFMS

#####################################
def generate_Chiplet_mapping(
    weights: list,
    X: int,
    NPE: int,
    NT: int,
    layers: int,
    allow_break_columns: bool = False,
    max_fill_percent: int = 100,
) -> list:

    # External helper must exist; preserving your call contract.
    tunable_params,xbars,IFMS,OFMS=calc_tunable_params(weights,X=X)

    # Build [cols, rows, total_xbars] for each logical layer (skipping weights[0] by your convention)
    XX = []
    for i, each in enumerate(weights):
        cols = each[5]
        rows = math.ceil(tunable_params[i] / max(cols, 1)) if cols > 0 else 0
        XX.append([cols, rows, int(xbars[i])])

    chip_capacity = NT * NPE
    usable_capacity = max(0, min(chip_capacity, math.floor(chip_capacity * (max_fill_percent / 100.0))))

    Chiplet = []
    remaining_usable = usable_capacity

    def new_chip():
        return {
            "Layers_filled": [],
            "Crossbars_filled_respective_layer": [],
            "Crossbars_remaining_respective_layer": [],
            "Layer_tile_distribution": {},  # layer_id: {tile_id: crossbars}
            "Empty_crossbars": chip_capacity  # will be corrected on finalize
        }

    def chip_used(blk):
        return sum(blk["Crossbars_filled_respective_layer"])

    def finalize_chip(blk):
        # True empty = total capacity - actually used (includes both reserved and unused usable)
        blk["Empty_crossbars"] = chip_capacity - chip_used(blk)

    chip = new_chip()

    # Tile tracking variables for current chiplet
    current_tile = 0
    current_tile_used = 0

    def add_layer_allocation(layer_num, crossbars_alloc, crossbars_remaining):
        nonlocal current_tile, current_tile_used

        # Check if adding this layer would exceed NT tiles - if so, we need new chiplet
        crossbars_to_place = crossbars_alloc
        temp_tile = current_tile
        temp_used = current_tile_used

        # Calculate how many tiles this allocation would need
        remaining_crossbars = crossbars_to_place
        max_tile_needed = temp_tile

        while remaining_crossbars > 0:
            space_in_current_tile = NPE - temp_used
            if remaining_crossbars <= space_in_current_tile:
                # Fits in current tile
                break
            else:
                # Need to move to next tile
                remaining_crossbars -= space_in_current_tile
                max_tile_needed += 1
                temp_used = 0

        # If would exceed NT tiles, return False to signal new chiplet needed
        if max_tile_needed >= NT:
            return False

        # Check if layer already exists in this chiplet
        if layer_num in chip["Layers_filled"]:
            # Layer already exists - merge the allocation
            layer_idx = chip["Layers_filled"].index(layer_num)
            chip["Crossbars_filled_respective_layer"][layer_idx] += crossbars_alloc
            chip["Crossbars_remaining_respective_layer"][layer_idx] = crossbars_remaining
        else:
            # New layer in this chiplet
            chip["Layers_filled"].append(layer_num)
            chip["Crossbars_filled_respective_layer"].append(crossbars_alloc)
            chip["Crossbars_remaining_respective_layer"].append(crossbars_remaining)

            # Initialize layer distribution
            chip["Layer_tile_distribution"][layer_num] = {}

        # Place crossbars across tiles
        remaining_crossbars = crossbars_to_place

        while remaining_crossbars > 0:
            space_in_current_tile = NPE - current_tile_used

            if remaining_crossbars <= space_in_current_tile:
                # Fits entirely in current tile
                if current_tile in chip["Layer_tile_distribution"][layer_num]:
                    chip["Layer_tile_distribution"][layer_num][current_tile] += remaining_crossbars
                else:
                    chip["Layer_tile_distribution"][layer_num][current_tile] = remaining_crossbars
                current_tile_used += remaining_crossbars
                remaining_crossbars = 0
            else:
                # Fill current tile and move to next
                if current_tile in chip["Layer_tile_distribution"][layer_num]:
                    chip["Layer_tile_distribution"][layer_num][current_tile] += space_in_current_tile
                else:
                    chip["Layer_tile_distribution"][layer_num][current_tile] = space_in_current_tile

                remaining_crossbars -= space_in_current_tile
                current_tile += 1
                current_tile_used = 0

        # If current tile is exactly full, move to next tile
        if current_tile_used == NPE:
            current_tile += 1
            current_tile_used = 0

        return True

    def reset_tile_tracking():
        nonlocal current_tile, current_tile_used
        current_tile = 0
        current_tile_used = 0

    i = 0
    layers_to_place = min(layers, len(XX))

    while i < layers_to_place:
        cols, rows, total_need = XX[i]
        remaining_need = total_need

        # Atomic column chunk if we split: keep column integrity
        # (number of crossbars needed per single column block across rows)
        atomic_chunk = math.ceil(rows / X) if cols > X and rows > 0 else total_need

        # If cols <= X: require full-layer fit; otherwise split by chunks/partial
        if cols <= X:
            # Not enough room in current chiplet -> roll to next chiplet
            if remaining_need > remaining_usable:
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()

            # Now it must fit (or usable is 0 → we just make an empty chiplet and move on)
            if remaining_need > remaining_usable:
                # Edge-case: usable capacity is 0 (max_fill_percent == 0) -> force new empty chiplet per layer
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()
                # If still zero, we just keep emitting empty chiplets; to avoid infinite loop, bail:
                # but better: place nothing and move on to next layer.
                i += 1
                continue

            # Place whole layer
            if not add_layer_allocation(i + 1, remaining_need, 0):
                # Need new chiplet due to tile limit
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()
                # Retry allocation in new chiplet
                add_layer_allocation(i + 1, remaining_need, 0)

            remaining_usable -= remaining_need
            i += 1

            # If usable area exactly full, close chiplet
            if remaining_usable == 0:
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()

            continue

        # cols > X: splitting allowed by chunks, maybe partial chunks if flag set
        while remaining_need > 0:
            # If nothing usable left in this chiplet, rotate
            if remaining_usable == 0:
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()

            alloc = 0
            if remaining_usable >= atomic_chunk:
                # Fit as many whole chunks as possible in one go
                k = min(remaining_usable // atomic_chunk, math.ceil(remaining_need / atomic_chunk))
                k = max(k, 1)
                alloc = min(k * atomic_chunk, remaining_need)
            else:
                # Not enough for a full chunk
                if allow_break_columns and remaining_usable > 0:
                    # Place a partial chunk (breaking a column) to consume leftover usable space
                    alloc = min(remaining_usable, remaining_need)
                else:
                    # Move to next chiplet; don't break columns here
                    finalize_chip(chip)
                    Chiplet.append(chip)
                    chip = new_chip()
                    remaining_usable = usable_capacity
                    reset_tile_tracking()
                    continue  # retry allocation in fresh chiplet

            # Commit allocation into current chiplet
            remaining_need -= alloc
            remaining_usable -= alloc

            if not add_layer_allocation(i + 1, alloc, max(remaining_need, 0)):
                # Need new chiplet due to tile limit
                finalize_chip(chip)
                Chiplet.append(chip)
                chip = new_chip()
                remaining_usable = usable_capacity
                reset_tile_tracking()
                # Retry allocation in new chiplet
                add_layer_allocation(i + 1, alloc, max(remaining_need, 0))
                remaining_need -= alloc
                remaining_usable -= alloc

            # If layer done, advance
            if remaining_need == 0:
                i += 1
                break

            # Otherwise loop continues; if usable hits zero, we'll roll to next chiplet at top

    # Flush last chiplet (even if empty usable reserved)
    if chip["Layers_filled"] or chip_used(chip) > 0 or usable_capacity < chip_capacity:
        finalize_chip(chip)
        Chiplet.append(chip)

    return Chiplet
##################################


In [248]:
#@title Tile to LIF/ LIF to tile traffic function
################################
def get_tile_to_lif_traffic(chiplet_data, layer_groups, layer_id,
                            tunable_params, xbars, weights, X,
                            Vmem_res=1, Timestep=1, bus_width=32,
                            use_tile_accumulators=False, use_TA_names=False):
    """
    Get traffic from individual tiles of a layer to the group's LIF.
    TA averages/sums by dividing by number of crossbars.
    """

    # Step 1: Find which group this layer belongs to
    lif_chiplet = None
    group_index = None
    for i, group_info in enumerate(layer_groups):
        layers = group_info[0]
        if layer_id in layers:
            lif_chiplet = group_info[1]
            group_index = i
            break

    if lif_chiplet is None:
        return [[0, "LIF_NOT_FOUND", 0, "NOT_FOUND", "NOT_FOUND"]]

    # Step 2: Create LIF and ACC names
    lif_name = f"LIF{group_index}"
    acc_name = f"ACC{group_index}"

    # Step 3: Calculate column information
    layer_idx = layer_id - 1
    output_channels = weights[layer_idx][5]  # N
    IFM_H = weights[layer_idx][0]
    IFM_W = weights[layer_idx][1]

    # Calculate total OFM from weights
    total_ofm = IFM_H * IFM_W * output_channels

    crossbars_per_column = int(math.ceil(tunable_params[layer_idx] / (X * output_channels)))
    total_crossbars = int(xbars[layer_idx])
    num_columns = int(total_crossbars / crossbars_per_column)

    # OFM per column
    ofm_per_column = total_ofm / num_columns
    traffic_per_column = math.ceil(ofm_per_column * Vmem_res * Timestep / bus_width)

    if total_crossbars == 0:
        return [[0, acc_name, 0, "NOT_FOUND", lif_chiplet],
                [acc_name, lif_name, 0, lif_chiplet, lif_chiplet]]

    # Step 5: Process tiles
    results = []
    global_crossbar_index = 0

    for chiplet_id, chiplet in enumerate(chiplet_data):
        if layer_id in chiplet.get('Layer_tile_distribution', {}):
            tile_distribution = chiplet['Layer_tile_distribution'][layer_id]

            for tile_id, tile_count in tile_distribution.items():
                # Determine columns in this tile
                tile_start = global_crossbar_index
                tile_end = global_crossbar_index + tile_count

                columns_in_tile = set()
                for cb_idx in range(tile_start, tile_end):
                    col_id = cb_idx // crossbars_per_column
                    columns_in_tile.add(col_id)

                num_columns_in_tile = len(columns_in_tile)

                # Traffic calculations
                crossbars_to_ta_traffic = tile_count * traffic_per_column

                # TA divides by number of crossbars it receives
                ta_to_acc_traffic = crossbars_to_ta_traffic // tile_count

                if use_tile_accumulators and tile_count > 1:
                    ta_identifier = f"TA{tile_id}" if use_TA_names else tile_id

                    results.append([tile_id, ta_identifier, crossbars_to_ta_traffic,
                                  chiplet_id, chiplet_id])
                    results.append([ta_identifier, acc_name, ta_to_acc_traffic,
                                  chiplet_id, lif_chiplet])
                else:
                    results.append([tile_id, acc_name, traffic_per_column,
                                  chiplet_id, lif_chiplet])

                global_crossbar_index += tile_count

    # Step 6: ACC -> LIF
    for col_id in range(num_columns):
        results.append([acc_name, lif_name, traffic_per_column, lif_chiplet, lif_chiplet])

    return results
################################
def get_lif_to_tile_traffic(chiplet_data, layer_groups, layer_output_sizes, layer_id,
                            tunable_params, xbars, weights, X,
                            Timestep=1, bus_width=32):
    """
    Get traffic from LIF directly to tiles (input traffic to the layer).

    LIF broadcasts input to tiles based on number of columns in each chiplet.
    Each column in a chiplet receives equal portion of input (divided by total columns).

    Args:
        chiplet_data: List of chiplet dictionaries
        layer_groups: List of [[layers], lif_chiplet_id]
        layer_output_sizes: Dictionary {layer_id: output_feature_map_size}
        layer_id: Which layer to check input tile traffic for
        tunable_params, xbars, weights, X: For column calculation

    Returns:
        List of [source, dest, traffic, from_chiplet, to_chiplet]
    """

    # Step 1: Find which group this layer belongs to and get previous layer
    target_group = None
    lif_chiplet = None
    group_index = None
    previous_layer = None
    previous_layer_lif_chiplet = None
    previous_group_index = None

    for i, group_info in enumerate(layer_groups):
        layers = group_info[0]
        if layer_id in layers:
            target_group = layers
            lif_chiplet = group_info[1]
            group_index = i
            layer_index = layers.index(layer_id)

            if layer_index > 0:
                previous_layer = layers[layer_index - 1]
                previous_layer_lif_chiplet = lif_chiplet
                previous_group_index = group_index
            else:
                previous_layer = layer_id - 1
                if previous_layer > 0:
                    for j, prev_group_info in enumerate(layer_groups):
                        prev_layers = prev_group_info[0]
                        if previous_layer in prev_layers:
                            previous_layer_lif_chiplet = prev_group_info[1]
                            previous_group_index = j
                            break
            break

    if target_group is None:
        return [[f"LIF{group_index}", 0, 0, "NOT_FOUND", "NOT_FOUND"]]

    if previous_layer is None or previous_layer <= 0:
        return [[f"LIF{group_index}", 0, 0, "NO_PREVIOUS", lif_chiplet]]

    if previous_layer_lif_chiplet is None:
        return [[f"LIF{group_index}", 0, 0, "PREV_LIF_NOT_FOUND", lif_chiplet]]

    # Step 2: Create LIF name
    source_lif_name = f"LIF{previous_group_index}"

    # Step 3: Get previous layer's OFM (current layer's IFM)
    previous_layer_ofm = layer_output_sizes.get(previous_layer, 0)

    # Step 4: Calculate current layer's column information
    layer_idx = layer_id - 1
    current_output_channels = weights[layer_idx][5]  # N

    current_crossbars_per_column = int(math.ceil(tunable_params[layer_idx] / (X * current_output_channels)))
    current_total_crossbars = int(xbars[layer_idx])
    current_num_columns = int(current_total_crossbars / current_crossbars_per_column)

    # Input traffic per column (divided by total number of columns)
    input_per_column = math.ceil(previous_layer_ofm / current_num_columns * Timestep / bus_width)

    # Step 5: Find which columns each chiplet/tile has
    results = []
    global_crossbar_index = 0

    for chiplet_id, chiplet in enumerate(chiplet_data):
        if layer_id in chiplet.get('Layer_tile_distribution', {}):
            tile_distribution = chiplet['Layer_tile_distribution'][layer_id]

            for tile_id, tile_count in tile_distribution.items():
                # Determine which columns this tile has
                tile_start = global_crossbar_index
                tile_end = global_crossbar_index + tile_count

                columns_in_tile = set()
                for cb_idx in range(tile_start, tile_end):
                    col_id = cb_idx // current_crossbars_per_column
                    columns_in_tile.add(col_id)

                num_columns_in_tile = len(columns_in_tile)

                # Traffic from LIF to this tile (based on columns it contains)
                tile_traffic = num_columns_in_tile * input_per_column

                # LIF → Tile (direct, no ACC/TA)
                results.append([source_lif_name, tile_id, tile_traffic,
                              previous_layer_lif_chiplet, chiplet_id])

                global_crossbar_index += tile_count

    return results
###############################


In [249]:
#@title Layer output/Layer input traffic chiplet level function
########################################################################################
def get_layer_input_traffic_chiplet(chiplet_data, layer_groups, layer_output_sizes, layer_id,
                                    tunable_params, xbars, weights, X,
                                    Timestep=1, buswidth=1):
    """
    Get input traffic from previous layer's LIF to current layer's chiplets.

    Each output column receives equal portion of input (divided by total number of columns).
    Chiplets receive traffic based on how many columns they contain.
    """

    # Step 1: Find which group this layer belongs to
    target_group = None
    lif_chiplet = None
    previous_layer = None
    previous_layer_lif_chiplet = None

    for group_info in layer_groups:
        layers = group_info[0]
        if layer_id in layers:
            target_group = layers
            lif_chiplet = group_info[1]
            layer_index = layers.index(layer_id)

            if layer_index > 0:
                previous_layer = layers[layer_index - 1]
                previous_layer_lif_chiplet = lif_chiplet
            else:
                previous_layer = layer_id - 1
                if previous_layer > 0:
                    for prev_group_info in layer_groups:
                        prev_layers = prev_group_info[0]
                        if previous_layer in prev_layers:
                            previous_layer_lif_chiplet = prev_group_info[1]
                            break
            break

    if target_group is None:
        return [[0, "NOT_FOUND", "NOT_FOUND"]]

    if previous_layer is None or previous_layer <= 0:
        return [[0, "NO_PREVIOUS", lif_chiplet]]

    if previous_layer_lif_chiplet is None:
        return [[0, "PREV_LIF_NOT_FOUND", lif_chiplet]]

    # Step 2: Get previous layer's OFM size
    previous_layer_ofm = layer_output_sizes.get(previous_layer, 0)

    # Step 3: Calculate current layer's column information
    layer_idx = layer_id - 1
    current_output_channels = weights[layer_idx][5]  # N

    current_crossbars_per_column = int(math.ceil(tunable_params[layer_idx] / (X * current_output_channels)))
    current_total_crossbars = int(xbars[layer_idx])
    current_num_columns = int(current_total_crossbars / current_crossbars_per_column)

    # Input traffic per column (divided by total number of columns)
    input_per_column = math.ceil(previous_layer_ofm / current_num_columns * Timestep / buswidth)

    # Step 4: Find which columns each chiplet has
    chiplet_columns = {}  # {chiplet_id: set of column_ids}
    global_crossbar_index = 0

    for chiplet_id, chiplet in enumerate(chiplet_data):
        if layer_id in chiplet.get('Layer_tile_distribution', {}):
            tile_distribution = chiplet['Layer_tile_distribution'][layer_id]

            for tile_id, tile_count in tile_distribution.items():
                tile_start = global_crossbar_index
                tile_end = global_crossbar_index + tile_count

                for cb_idx in range(tile_start, tile_end):
                    col_id = cb_idx // current_crossbars_per_column
                    if chiplet_id not in chiplet_columns:
                        chiplet_columns[chiplet_id] = set()
                    chiplet_columns[chiplet_id].add(col_id)

                global_crossbar_index += tile_count

    if len(chiplet_columns) == 0:
        return [[0, previous_layer_lif_chiplet, "NOT_FOUND"]]

    results = []

    # Step 5: Calculate traffic per chiplet based on columns it contains
    for chiplet_id, columns in chiplet_columns.items():
        num_columns_in_chiplet = len(columns)

        # Each column receives input_per_column
        chiplet_traffic = num_columns_in_chiplet * input_per_column

        results.append([chiplet_traffic, previous_layer_lif_chiplet, chiplet_id])

    return results
########################################################################################
def get_layer_output_traffic_chiplet(chiplet_data, layer_groups, layer_output_sizes, layer_id,
                                     tunable_params, xbars, weights, X,
                                     Vmem_res=1, Timestep=1, buswidth=1):
    """
    Get traffic from chiplets to LIF, including ACC breakdown with column awareness.

    Returns:
        List of traffic entries: [traffic, source_chiplet, dest_chiplet]
    """
    # Step 1: Find which chiplet each layer is in and tile distribution
    layer_chiplet_distribution = {}
    for chiplet_id, chiplet in enumerate(chiplet_data):
        if layer_id in chiplet.get('Layer_tile_distribution', {}):
            tile_distribution = chiplet['Layer_tile_distribution'][layer_id]
            num_tiles = len(tile_distribution)
            layer_chiplet_distribution[chiplet_id] = num_tiles

    # Step 2: Find which group this layer belongs to
    lif_chiplet = None
    group_index = None
    for i, group_info in enumerate(layer_groups):
        layers = group_info[0]
        if layer_id in layers:
            lif_chiplet = group_info[1]
            group_index = i
            break

    if lif_chiplet is None:
        return [[0, "NOT_FOUND", "NOT_FOUND"]]

    # Step 3: Calculate column information
    layer_idx = layer_id - 1
    output_channels = weights[layer_idx][5]
    IFM_H = weights[layer_idx][0]
    IFM_W = weights[layer_idx][1]

    total_ofm = IFM_H * IFM_W * output_channels

    crossbars_per_column = int(math.ceil(tunable_params[layer_idx] / (X * output_channels)))
    total_crossbars = int(xbars[layer_idx])
    num_columns = int(total_crossbars / crossbars_per_column)

    # OFM per column
    ofm_per_column = total_ofm / num_columns
    traffic_per_column = math.ceil(ofm_per_column * Vmem_res * Timestep / buswidth)

    if len(layer_chiplet_distribution) == 0:
        return [[0, "NOT_FOUND", lif_chiplet]]

    results = []

    # Step 4: Calculate traffic from each chiplet to ACC
    for chiplet_id, num_tiles in layer_chiplet_distribution.items():
        # Each tile sends one column's worth of traffic
        chiplet_traffic = num_tiles * traffic_per_column
        results.append([chiplet_traffic, chiplet_id, lif_chiplet])

    # Step 5: Add ACC -> LIF traffic (per column)
    for col_id in range(num_columns):
        results.append([traffic_per_column, lif_chiplet, lif_chiplet])

    return results



In [250]:
#@title System Chiplet level matrix function
import pandas as pd
import numpy as np

def create_chiplet_matrix(results_list, N):
    # Initialize N x N matrix with zeros
    matrix = np.zeros((N, N), dtype=int)

    # Flatten the results_list and process each result tuple
    for result_layer in results_list:
        for data_flow, from_chiplet, to_chiplet in result_layer:
            # Only process if both chiplets are within the N range
            if from_chiplet < N and to_chiplet < N:
                # Add data flow to the matrix (accumulate if duplicate entries exist)
                matrix[from_chiplet][to_chiplet] += data_flow

    # Create chiplet names (C0, C1, C2, ..., CN-1)
    chiplet_names = [f'C{i}' for i in range(N)]

    # Convert to pandas DataFrame with named rows and columns
    df = pd.DataFrame(matrix, index=chiplet_names, columns=chiplet_names)

    return matrix,df

In [251]:
#@title TIle level matrix function
import numpy as np
import pandas as pd

def create_tile_matrix(results_list, target_chiplet, layer_groups, NT, total_chiplets, use_tile_accumulators=False, use_TA_names=False):
    """
    Create traffic matrix for a specific chiplet showing tile-level traffic.

    Updated to handle:
    - TA (Tile Accumulators): within-tile crossbar accumulation
    - ACC (LIF Accumulators): cross-tile/chiplet accumulation
    - Kernel position partitioning traffic patterns

    Args:
        results_list: List of traffic results from all layers
        target_chiplet: Which chiplet to visualize
        layer_groups: List of [[layers], lif_chiplet_id]
        NT: Number of tiles
        total_chiplets: Total number of chiplets
        use_tile_accumulators: If True, include TA nodes
        use_TA_names: If True, use "TA0" names; if False, use tile numbers for TAs
    """

    # Create tile labels
    tile_labels = [f"T{i}" for i in range(NT)]

    # Find which LIFs and ACCs are located in the target chiplet
    target_lifs = []
    target_accs = []
    target_tas = []

    # Collect TAs if using tile accumulators
    if use_tile_accumulators:
        for i in range(NT):
            if use_TA_names:
                target_tas.append(f"TA{i}")
            # If not use_TA_names, TAs are represented by tile numbers, already in tile_labels

    # Collect LIFs and ACCs for this chiplet
    for i, (layers, lif_chiplet) in enumerate(layer_groups):
        if lif_chiplet == target_chiplet:
            target_accs.append(f"ACC{i}")
            target_lifs.append(f"LIF{i}")

    # Create other chiplet labels (exclude target chiplet)
    other_chiplet_labels = [f"C{i}" for i in range(total_chiplets) if i != target_chiplet]

    # Combine all labels based on flags
    if use_tile_accumulators and use_TA_names:
        # Tiles -> TAs -> ACCs -> LIFs -> Other Chiplets
        all_labels = tile_labels + target_tas + target_accs + target_lifs + other_chiplet_labels
    else:
        # Tiles -> ACCs -> LIFs -> Other Chiplets
        all_labels = tile_labels + target_accs + target_lifs + other_chiplet_labels

    matrix_size = len(all_labels)

    # Initialize matrix
    matrix = np.zeros((matrix_size, matrix_size), dtype=int)

    # Create label to index mapping
    label_to_idx = {label: i for i, label in enumerate(all_labels)}

    # Process each layer's traffic results
    for result_layer in results_list:
        for traffic_entry in result_layer:
            if len(traffic_entry) >= 5:
                source, dest, traffic, source_chiplet, dest_chiplet = traffic_entry[:5]

                # Only process traffic involving the target chiplet
                if source_chiplet == target_chiplet or dest_chiplet == target_chiplet:

                    # Determine source and destination labels
                    source_label = None
                    dest_label = None

                    # Handle source
                    if isinstance(source, int):  # Tile
                        if source_chiplet == target_chiplet:
                            source_label = f"T{source}"
                        else:
                            source_label = f"C{source_chiplet}"
                    elif isinstance(source, str):
                        if source.startswith("TA"):  # Tile Accumulator
                            if source_chiplet == target_chiplet:
                                source_label = source
                            else:
                                source_label = f"C{source_chiplet}"
                        elif source.startswith("ACC"):  # LIF Accumulator
                            if source_chiplet == target_chiplet:
                                source_label = source
                            else:
                                source_label = f"C{source_chiplet}"
                        elif source.startswith("LIF"):  # LIF
                            if source_chiplet == target_chiplet:
                                source_label = source
                            else:
                                source_label = f"C{source_chiplet}"

                    # Handle destination
                    if isinstance(dest, int):  # Tile
                        if dest_chiplet == target_chiplet:
                            dest_label = f"T{dest}"
                        else:
                            dest_label = f"C{dest_chiplet}"
                    elif isinstance(dest, str):
                        if dest.startswith("TA"):  # Tile Accumulator
                            if dest_chiplet == target_chiplet:
                                dest_label = dest
                            else:
                                dest_label = f"C{dest_chiplet}"
                        elif dest.startswith("ACC"):  # LIF Accumulator
                            if dest_chiplet == target_chiplet:
                                dest_label = dest
                            else:
                                dest_label = f"C{dest_chiplet}"
                        elif dest.startswith("LIF"):  # LIF
                            if dest_chiplet == target_chiplet:
                                dest_label = dest
                            else:
                                dest_label = f"C{dest_chiplet}"

                    # Add traffic to matrix if both labels are valid
                    if (source_label and dest_label and
                        source_label in label_to_idx and dest_label in label_to_idx):
                        src_idx = label_to_idx[source_label]
                        dest_idx = label_to_idx[dest_label]
                        matrix[src_idx][dest_idx] += traffic

    # Create pandas DataFrame
    df = pd.DataFrame(matrix, index=all_labels, columns=all_labels)

    return matrix, df

In [252]:
#@title Define Layers, H/W param, Layer Group with LIF
## IFM length, IFM Width, IFM Channel Depth, Kernel Length, Kernel Width, Kernel Depth, Pool/Not, Stride
weights=[
    (32,32,3,3,3, 3,0,1),
    (32,32,3,3,3,64,0,1),
    (32,32,64,3,3,128,1,1), #pool here
    (16,16,128,3,3,128,0,1),
    (16,16,128,3,3,256,1,1),#pool here
    (8,8,256,3,3,256,0,1),
    (8,8,256,3,3,256,0,1),
    (8,8,256,3,3,512,1,1),#pool here
    (4,4,512,3,3,512,0,1),
    (4,4,512,3,3,512,0,1),
    (4,4,512,3,3,512,1,1),#pool here
    (2,2,512,3,3,512,0,1),
    (2,2,512,3,3,512,0,1),
    (2,2,512,3,3,512,1,1),#pool here
    (1,1,512,1,1,1000,0,1)
]

# layer_groups = [
#     [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 2]
# ]

layer_groups = [
        [[1], 0],      # Layer 1 - ends in chiplet 0
        [[2], 0],      # Layer 2 - ends in chiplet 0
        [[3], 0],      # Layer 3 - ends in chiplet 0
        [[4], 0],      # Layer 4 - ends in chiplet 0
        [[5], 0],      # Layer 5 - ends in chiplet 0 (crossbar 1)
        [[6], 0],      # Layer 6 - ends in chiplet 0 (crossbar 3)
        [[7], 0],      # Layer 7 - ends in chiplet 0 (crossbar 4)
        [[8], 0],      # Layer 8 - ends in chiplet 0 (crossbar 7)
        [[9], 0],      # Layer 9 - ends in chiplet 0 (crossbar 13)
        [[10], 1],     # Layer 10 - ends in chiplet 1 (crossbar 15)
        [[11], 1],     # Layer 11 - ends in chiplet 1 (crossbar 9)
        [[12], 1],     # Layer 12 - ends in chiplet 1 (crossbar 15)
        [[13], 2],     # Layer 13 - ends in chiplet 2 (crossbar 5)
        [[14], 2],     # Layer 14 - ends in chiplet 2 (crossbar 11)
        [[15], 2]      # Layer 15 - ends in chiplet 2 (crossbar 12)
    ]

NPE=19
NT=16
P = 100
X = 128

Vmem_res=4
Timestep=5

NoC_buswidth=32
NoI_buswidth=32

allow_break_columns=True

In [253]:
#@title Generate System & Chiplet traffic data

# External helper must exist; preserving your call contract.
tunable_params,xbars,IFMS,OFMS=calc_tunable_params(weights,X=X)
layer_output_sizes = dict(zip(range(1,len(OFMS)+1), OFMS))
Chiplet = generate_Chiplet_mapping(weights, X=X, NPE=NPE, NT=NT, layers=len(weights), allow_break_columns=allow_break_columns, max_fill_percent=P)
#######################################################
# Combine all layers traffic at both Chiplet and Tile levels
all_chiplet_result = []
all_tile_result = []

for layer in layer_output_sizes.keys():
    # Process chiplet traffic
    o_chiplet=get_layer_output_traffic_chiplet(Chiplet, layer_groups, layer_output_sizes, layer,
                                 tunable_params=tunable_params, xbars=xbars,weights=weights,X=X,
                                 Vmem_res=Vmem_res, Timestep=Timestep, buswidth=NoI_buswidth)
    # Process tile traffic
    o_tile = get_tile_to_lif_traffic(Chiplet, layer_groups, layer,
                        tunable_params=tunable_params, xbars=xbars,weights=weights,X=X,
                        Vmem_res=Vmem_res, Timestep=Timestep, bus_width=NoC_buswidth,use_tile_accumulators=True,use_TA_names=False)
    if layer > 1:
        # Process input traffic for both chiplet and tile
        i_chiplet = get_layer_input_traffic_chiplet(Chiplet, layer_groups, layer_output_sizes, layer,
                                tunable_params=tunable_params, xbars=xbars,weights=weights,X=X,
                                Timestep=Timestep, buswidth=NoI_buswidth)
        i_tile = get_lif_to_tile_traffic(Chiplet, layer_groups, layer_output_sizes, layer,
                        tunable_params=tunable_params, xbars=xbars,weights=weights,X=X,
                        Timestep=Timestep, bus_width=NoC_buswidth)
        all_chiplet_result.append(i_chiplet)
        all_tile_result.append(i_tile)

    all_chiplet_result.append(o_chiplet)
    all_tile_result.append(o_tile)

# display(all_chiplet_result)
# display(all_tile_result)

In [254]:
#@title Print System Overall Requirements
print("Total no of layers:"+ str(len(weights)))
print("No of crossbars per Chiplet:"+ str(NT*NPE))


print("Total no of chiplets:", max(max(tuple[1] for tuple in sublist) for sublist in all_chiplet_result)+1)

print("layer_output_sizes: (bits)",layer_output_sizes,'\n',end='='*100)
# display(Chiplet)

Total no of layers:15
No of crossbars per Chiplet:304
Total no of chiplets: 4
layer_output_sizes: (bits) {1: 3072, 2: 65536, 3: 131072, 4: 32768, 5: 65536, 6: 16384, 7: 16384, 8: 32768, 9: 8192, 10: 8192, 11: 8192, 12: 2048, 13: 2048, 14: 2048, 15: 1000} 

In [255]:
#@title Print System traffic matrix
##create Chiplet traffic matrix
N = max(max(tuple[1] for tuple in sublist) for sublist in all_chiplet_result)+1 ##finding last chiplet
matrix_system, df_system=create_chiplet_matrix(all_chiplet_result, N)
print('='*80,'\nSystem Chiplet level traffic\n',end='='*80)

html = df_system.replace(0, '*').to_html()
styled_html = f"""<div style="font-size: 14px; font-family: monospace;">{html}</div>"""
display(HTML(styled_html))


System Chiplet level traffic

,C0,C1,C2,C3
C0,540640,2560,*,*
C1,1280,34560,1920,*
C2,*,3840,6472,640
C3,*,*,2477,*


In [256]:
#@title Print Chiplet traffic matrix(s)
# display(all_tile_result)
# N = max(max(tuple[1] for tuple in sublist) for sublist in all_tile_result)+1 ##finding last chiplet
all_chiplet=[]
include_chiplets=True
for target_chiplet in range(N):
  matrix, df = create_tile_matrix(all_tile_result,
                                      target_chiplet=target_chiplet,
                                      layer_groups=layer_groups,
                                      NT=NT, total_chiplets=N if include_chiplets else 0
                                      )
  all_chiplet.append([matrix,df])


# Display loop with styling
for target_chiplet in range(N):
  print('='*80)
  print("Chiplet",target_chiplet,'tile traffic')
  print('='*80)
  df = all_chiplet[target_chiplet][1]
  # Apply styling and display as HTML

  html = df.replace(0, '*').to_html()
  styled_html = f"""<div style="font-size: 5px; font-family: monospace;">{html}</div>"""
  display(HTML(styled_html))
  # display(data_table.DataTable(all_chiplet[target_chiplet][1],num_rows_per_page=100,max_columns=None))

Chiplet 0 tile traffic


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,ACC0,ACC1,ACC2,ACC3,ACC4,ACC5,ACC6,ACC7,ACC8,LIF0,LIF1,LIF2,LIF3,LIF4,LIF5,LIF6,LIF7,LIF8,C1,C2,C3
T0,655360,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,1920,40960,81920,20480,20480,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
T1,*,327680,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,20480,5120,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
T2,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
T3,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,5120,*,*,*,*,*,*,*,*,*,*,*,*,*,*
T4,*,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,*,*,*,*,*,*,*,*,*,*,*,*,*,*
T5,*,*,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,5120,*,*,*,*,*,*,*,*,*,*,*,*,*
T6,*,*,*,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,*,*,*,*,*,*,*,*,*,*,*,*,*
T7,*,*,*,*,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,*,*,*,*,*,*,*,*,*,*,*,*,*
T8,*,*,*,*,*,*,*,*,97280,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,*,*,*,*,*,*,*,*,*,*,*,*,*
T9,*,*,*,*,*,*,*,*,*,51200,*,*,*,*,*,*,*,*,*,*,*,*,*,5120,1280,*,*,*,*,*,*,*,*,*,*,*,*


Chiplet 1 tile traffic


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,ACC9,ACC10,ACC11,LIF9,LIF10,LIF11,C0,C2,C3
T0,23040,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,1280,*,*
T1,*,24320,*,*,*,*,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T2,*,*,24320,*,*,*,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T3,*,*,*,24320,*,*,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T4,*,*,*,*,24320,*,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T5,*,*,*,*,*,24320,*,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T6,*,*,*,*,*,*,24320,*,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T7,*,*,*,*,*,*,*,24320,*,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*,*
T8,*,*,*,*,*,*,*,*,24320,*,*,*,*,*,*,*,1280,1280,*,*,*,*,*,*,*
T9,*,*,*,*,*,*,*,*,*,24320,*,*,*,*,*,*,*,1280,*,*,*,*,*,*,*


Chiplet 2 tile traffic


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,ACC12,ACC13,ACC14,LIF12,LIF13,LIF14,C0,C1,C3
T0,8000,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,1600,*
T1,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T2,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T3,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T4,*,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T5,*,*,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T6,*,*,*,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320,*
T7,*,*,*,*,*,*,*,6080,*,*,*,*,*,*,*,*,320,*,*,*,*,*,*,320,*
T8,*,*,*,*,*,*,*,*,6080,*,*,*,*,*,*,*,320,*,*,*,*,*,*,*,*
T9,*,*,*,*,*,*,*,*,*,6080,*,*,*,*,*,*,320,*,*,*,*,*,*,*,*


Chiplet 3 tile traffic


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,C0,C1,C2
T0,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320
T1,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320
T2,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320
T3,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,*,320
T4,*,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,*,320
T5,*,*,*,*,*,6080,*,*,*,*,*,*,*,*,*,*,*,*,320
T6,*,*,*,*,*,*,5357,*,*,*,*,*,*,*,*,*,*,*,399
T7,*,*,*,*,*,*,*,1501,*,*,*,*,*,*,*,*,*,*,79
T8,*,*,*,*,*,*,*,*,790,*,*,*,*,*,*,*,*,*,79
T9,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [257]:
# Calculate total NoI traffic from the system matrix (excluding diagonal)
total_NoI_traffic = matrix_system[~np.eye(matrix_system.shape[0], dtype=bool)].sum()

print("Total NoI traffic:", total_NoI_traffic)

Total NoI traffic: 12717
